In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [9]:
class State(TypedDict):
    messages:Annotated[List, "List of messages in the conversation"]
    # messages: List[Dict[str, Any]]
    # user_input: str
    # response: str

graph_builder = StateGraph(State)

In [10]:
graph_builder

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

False